In [2]:
print ('Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto')

Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto


In [7]:
# import first all required dependencies for the project
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

from bs4 import BeautifulSoup

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [6]:
# getting and importing wikipedia page for toronta postal code
wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
wiki_html = requests.get(wiki_url)
wiki_html.text[:100]

'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title'

In [8]:
# using bsoup for reading wiki page
soup = BeautifulSoup(wiki_html.text, 'lxml')
wikitable = soup.find_all('table')[0] 
wikitable

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

In [15]:
# creating df table for first assignment

Toronto_table = pd.DataFrame()

row_tr = 0

for row in wikitable.find_all('tr'):
    column_td = 0
    columns = row.find_all('td')
    for column in columns:
        Toronto_table.loc[row_tr,column_td] = column.get_text()
        column_td += 1
    row_tr += 1
Toronto_table

,0,1,2
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n
5,M5A,Downtown Toronto,Harbourfront\n
6,M5A,Downtown Toronto,Regent Park\n
7,M6A,North York,Lawrence Heights\n
8,M6A,North York,Lawrence Manor\n
9,M7A,Queen's Park,Not assigned\n
10,M8A,Not assigned,Not assigned\n


In [32]:
#changing the column name headings for the table
Toronto_table.rename(columns = {0 : 'PostalCode', 1:'Borough', 2: 'Neighborhood'}, inplace = True)
#data cleaning 
Toronto_table = Toronto_table[Toronto_table.Borough!='Not assigned']
Toronto_table.reset_index(drop=True, inplace= True)
Toronto_table['Neighborhood'] =Toronto_table['Neighborhood'].str.replace('\n', '')

# Queens park has no assigned value for neighborhood so arrange this data
Toronto_table[Toronto_table.Neighborhood == 'Not assigned']
index_neighb_na = Toronto_table[Toronto_table.Neighborhood == 'Not assigned'].index
value_neighb_na = Toronto_table.iloc[index_neighb_na, 1]
Toronto_table = Toronto_table.set_value (index_neighb_na,'Neighborhood', value_neighb_na)

# combine the unique data for postalcodes as requested
unique_Boroughs=Toronto_table.groupby('PostalCode').nunique()
print('Unique borough number is', unique_Boroughs.shape)

Toronto_table['Neighborhood'] = Toronto_table[['PostalCode','Borough','Neighborhood']].groupby(['PostalCode','Borough'])['Neighborhood'].transform(lambda x: ', '.join(x)) 
Toronto_table.drop_duplicates(inplace=True)
Toronto_table.reset_index(drop=True, inplace=True)
Toronto_table


Unique borough number is (103, 3)


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:12: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park, Harbourfront, Regen..."
3,M6A,North York,"Lawrence Heights, Lawrence Manor, Lawrence Hei..."
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern, Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill, Woodbine Gard..."
9,M5B,Downtown Toronto,"Ryerson, Garden District, Ryerson, Garden Dist..."


In [34]:
Toronto_table.head()
Toronto_table.shape

(103, 3)

In [35]:
Toronto_table.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park, Harbourfront, Regen..."
3,M6A,North York,"Lawrence Heights, Lawrence Manor, Lawrence Hei..."
4,M7A,Queen's Park,Queen's Park
